In [4]:
# train.py - 使用你定义的 YOLODataset + YOLOTiny + YOLOLoss 进行训练
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from dataset import YOLODataset
from model import YOLOTiny
from loss import YOLOLoss

# 参数配置
IMG_SIZE = 416
NUM_CLASSES = 13
ANCHORS = 3
BATCH_SIZE = 8
EPOCHS = 50
LR = 1e-4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据路径
img_dir = "D:/PycharmProjects/EE-451-project/dataset_project_iapr2025/train"
label_dir = "/project1/data/obj_train_data"

# 数据加载
train_dataset = YOLODataset(img_dir, label_dir, img_size=IMG_SIZE)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

# 模型、优化器、损失函数
model = YOLOTiny(num_classes=NUM_CLASSES, anchors=ANCHORS).to(device)
optimizer = optim.Adam(model.parameters(), lr=LR)
criterion = YOLOLoss(S=13, B=ANCHORS, num_classes=NUM_CLASSES)

# 训练循环
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for imgs, targets in train_loader:
        imgs = imgs.to(device)
        targets = targets.to(device)
        preds = model(imgs)
        loss = criterion(preds, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {total_loss / len(train_loader):.4f}")

# 保存模型
torch.save(model.state_dict(), "yolo_tiny_trained.pth")

ValueError: num_samples should be a positive integer value, but got num_samples=0